In [1]:
# 1.导入相关依赖
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()
# 2.定义文档
document_1 = Document(
    page_content="经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！",
)
document_2 = Document(
    page_content="基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。",
)
document_3 = Document(page_content="气候变化：承诺到2030年将温室气体排放量减少50%。",
                      )
document_4 = Document(
    page_content=" 医疗保健：降低处方药价格，扩大医疗保险覆盖范围。",
)
document_5 = Document(
    page_content="教育：提供免费的社区大学教育。。",
)
document_6 = Document(
    page_content="科技：增加对半导体产业的投资以减少对外国供应链的依赖。。",
)
document_7 = Document(
    page_content="外交政策：继续支持乌克兰对抗俄罗斯的侵略。",
)
document_8 = Document(
    page_content="枪支管制：呼吁国会通过更严格的枪支管制法律。",
)
document_9 = Document(
    page_content="移民改革：提出全面的移民改革方案。",
)
document_10 = Document(
    page_content="社会正义：承诺解决系统性种族歧视问题。",
)
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
# 3.创建向量存储
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)
# 4.将文档向量化，添加到向量数据库索引中，得到向量数据库对象
db = FAISS.from_documents(documents, embeddings)

默认检索器使用相似性搜索

In [6]:
# 获取检索器
retriever = db.as_retriever(
    search_kwargs={"k": 3}
)  #这里设置返回的文档数
docs = retriever.invoke("经济政策")
for i, doc in enumerate(docs):
    print(f"\n结果 {i + 1}:\n{doc.page_content}\n")


结果 1:
经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


结果 2:
科技：增加对半导体产业的投资以减少对外国供应链的依赖。。


结果 3:
外交政策：继续支持乌克兰对抗俄罗斯的侵略。



分数阈值查询

In [3]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)
docs = retriever.invoke("经济政策")
for doc in docs:
    print(f"📌 内容: {doc.page_content}")

📌 内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


 MMR搜索

In [8]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"fetch_k": 2}
)
docs = retriever.invoke("经济政策")
print(len(docs))
for doc in docs:
    print(f"📌 内容: {doc.page_content}")

2
📌 内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！
📌 内容: 科技：增加对半导体产业的投资以减少对外国供应链的依赖。。


# 3、结合大模型的使用
举例1：不实用RAG技术

In [9]:
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()
# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")
# 调用
response = llm.invoke("北京有什么著名的建筑？")
print(response.content)

北京作为中国的首都，有许多著名的建筑，以下是一些代表性建筑：

1. **故宫**：又称紫禁城，是明清两代的皇宫，拥有壮丽的古代建筑和丰富的历史文化。

2. **天安门广场**：世界上最大的城市广场之一，周围有人民英雄纪念碑、毛主席纪念堂等重要建筑。

3. **颐和园**：中国最大的皇家园林，以其精美的湖泊、山丘和古典建筑而闻名。

4. **长城**：有多个段落在北京附近，如八达岭和慕田峪，是中国古代防御工程的象征。

5. **天坛**：明清时期皇帝祭天的地方，以其独特的建筑风格和壮丽的园林著称。

6. **鸟巢**：国家体育场，因其独特的设计而闻名，是2008年北京奥运会的主要场馆。

7. **水立方**：国家游泳中心，以其独特的水泡形状和现代建筑风格而备受瞩目。

8. **北京大学和清华大学**：这两所大学不仅学术声誉卓著，还拥有许多美丽的校园建筑。

9. **中央电视台总部大楼**：以独特的“弯曲”形状而著称，是现代北京建筑的代表之一。

10. **大钟寺**：以其巨大的铜钟和古老的建筑群而闻名，是北京的重要文化遗址。

这些建筑代表了北京悠久的历史和丰富的文化，是游客和当地居民生活中不可或缺的一部分。


举例2: 通过FAISS构建一个可搜索的向量索引数据库，并结合RAG技术让LLM去回答问题

In [11]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
import dotenv

dotenv.load_dotenv()
# 2. 创建自定义提示词模板
prompt_template = """请使用以下提供的文本内容来回答问题。仅使用提供的文本信息，如果文本中
没有相关信息，请回答"抱歉，提供的文本中没有这个信息"。
文本内容：
{context}
问题：{question}
回答：
"
"""
prompt = PromptTemplate.from_template(prompt_template)
# 3. 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
# 4. 加载文档
loader = TextLoader("./10-ai.txt", encoding='utf-8')
documents = loader.load()
# 5. 分割文档
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
texts = text_splitter.split_documents(documents)
#print(f"文档个数:{len(texts)}")
# 6. 创建向量存储
vectorstore = FAISS.from_documents(
    documents=texts,
    embedding=embedding_model
)
# 7.获取检索器
retriever = vectorstore.as_retriever()
docs = retriever.invoke("北京有什么著名的建筑？")
# 8. 创建Runnable链
chain = prompt | llm
# 9. 提问
result = chain.invoke(input={"question": "北京有什么著名的建筑？", "context": docs})
print("\n回答:", result.content)


回答: 北京的著名建筑包括：

1. 故宫（紫禁城）：明清两代的皇家宫殿，是世界上现存规模最大、保存最完整的古代木结构建筑群，现为故宫博物院，收藏大量珍贵文物。
2. 天坛：明清皇帝祭天祈谷的场所，以祈年殿为代表，建筑布局严谨，体现中国古代“天人合一”的哲学思想。
3. 长城（八达岭、慕田峪段）：虽不位于市中心，但北京境内的长城段落最为著名，是世界文化遗产，象征中华民族的坚韧精神。
4. 颐和园：中国现存最大的皇家园林，以昆明湖、万寿山为基础，融合江南园林风格与北方山水特色。
5. 国家体育场（“鸟巢”）：2008年北京奥运会主会场，由钢结构编织而成，外形独特，现为大型体育赛事和文化活动举办地。
6. 国家游泳中心（“水立方”）：奥运会游泳场馆，采用ETFE膜结构，夜晚灯光绚丽，后改造为“冰立方”用于冬奥会。
7. 中央广播电视塔：位于海淀区，高405米，曾是北京最高建筑之一，可俯瞰城市全景。
8. 中国尊（中信大厦）：位于北京CBD，高528米，是目前北京最高的摩天大楼，外形灵感源自中国古代礼器“尊”。
